RAG Configuration in this notebook:

Embedding model: sentence-transformers/all-mpnet-base-v2

Chunk size: 1000

Chunk overlap: 100

Generation Model: llama-3-8B-Instruct

Retriever: VectorStore

Embedding Size: 768

This notebook also includes RAG response generation and evaluation.

In [ ]:
import pickle
from pinecone import Pinecone, ServerlessSpec
import getpass
import os
import time
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import pinecone
import torch
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pinecone import Index
from langchain_pinecone import PineconeVectorStore
from transformers import BitsAndBytesConfig
import bitsandbytes
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
from tqdm import tqdm
from langchain.prompts import PromptTemplate
import pandas as pd

In [2]:
if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

Enter your Pinecone API key: ··········


In [ ]:
#Embedding model
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda" if torch.cuda.is_available() else "cpu"}
encode_kwargs = {"normalize_embeddings": False}
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

<ipython-input-5-37995c8f00a0>:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
# Create the index if it doesn't exist
#pc.create_index(name="rag-llm-1000",
#                    dimension=768,metric="cosine",
#                   spec=ServerlessSpec(
#                    cloud="aws",
#                    region="us-east-1"))

pc_index = pc.Index("rag-llm-1000")

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Load the document
with open("data_5983_updated.pkl", "rb") as file:
    documents = pickle.load(file)

In [ ]:
# Chunking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)

chunks = text_splitter.split_documents(documents)


In [ ]:
len(chunks)

2644

In [ ]:
total_size = sum(len(chunk.page_content) for chunk in chunks)
average_size = total_size / len(chunks) if chunks else 0

print(f"Total size of chunks: {total_size}")
print(f"Number of chunks: {len(chunks)}")
print(f"Average size of chunks: {average_size:.2f} characters")


Total size of chunks: 1669472
Number of chunks: 2644
Average size of chunks: 631.42 characters


In [ ]:
from langchain_pinecone import PineconeVectorStore

# VectorStore
vectorstore = PineconeVectorStore(
    index_name="rag-llm-1000",
    embedding=hf_embeddings,
)

In [ ]:
#for chunk in tqdm(chunks, desc="Adding documents to Pinecone", unit="chunk"):
#    vectorstore.add_documents([chunk])

Adding documents to Pinecone: 100%|██████████| 2644/2644 [13:22<00:00,  3.30chunk/s]


In [4]:
index_stats = pc_index.describe_index_stats()

print("Index Stats:", index_stats)

Index Stats: {'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 2644}},
 'total_vector_count': 2644}


In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [ ]:
!pip install -U bitsandbytes

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import bitsandbytes

bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)

llm_model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", quantization_config=bnb_config)
llm_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [ ]:
llm_tokenizer.pad_token_id = llm_tokenizer.eos_token_id

In [ ]:
llm_pipeline = pipeline(
    "text-generation",
    model=llm_model,
    tokenizer=llm_tokenizer,
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=100,
    top_p=0.9,
    top_k=50,
    eos_token_id=llm_tokenizer.eos_token_id
)

In [ ]:
from langchain.llms import HuggingFacePipeline

llm_final_model = HuggingFacePipeline(pipeline=llm_pipeline)

<ipython-input-18-3f741d8043ea>:3: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm_final_model = HuggingFacePipeline(pipeline=llm_pipeline)


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# Prompt template
template = """
You are a compassionate and knowledgeable mental health assistant that answers questions related to mental health.\n
Use the following pieces of retrieved context to provide a helpful and empathetic response to the user's question.\n
Use only the context provided and not any prior knowledge.\n
If you are unsure of the answer, tell that you do not know the answer.\n
Stick to the question and just answer the question in a short manner.\n
Avoid any additional greetings or elaborations.\n

Context: \n
------------------------------------------------------------------------------\n
{context}
------------------------------------------------------------------------------\n
Given the context and without any prior knowledge, answer the below question.\n
Question: {question}
Answer:
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm_final_model,
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
query = "What is the Four-Fold Breath technique?"

result= qa_chain.invoke(query)

In [ ]:
answer = result['result']
print("Answer:", answer)

Answer: The Four-Fold Breath technique involves breathing in slowly to the count of 4 and breathing out even slower to the count of 6. This exercise is recommended to be practiced for about 5 minutes, three times a day, focusing solely on your breathing.


RAG Response Generation

In [ ]:
df = pd.read_csv("U_eval_dataset_100q_with_context.csv")

In [ ]:
df.head()

,Question,Ground_Truth,Source,Category,contexts
0,What triggers Seasonal Affective Disorder (SAD)?,"SAD is triggered by seasonal changes, especial...",https://www.samhsa.gov/mental-health/seasonal-...,Mental Illness,"['Seasonal affective disorder, or SAD, is a co..."
1,What are the health risks of bulimia nervosa?,"Risks include dental decay, dehydration, and s...",https://www.samhsa.gov/mental-health/eating-di...,Mental Illness,['It is common that binge eating will lead to ...
2,How can someone with mental health problems ma...,By finding trustworthy individuals who provide...,https://www.samhsa.gov/mental-health/how-to-ta...,Mental Health,"[""Anyone can experience mental health problems..."
3,What are the risk factors for schizophrenia?,Risk factors include genetic predisposition an...,https://www.samhsa.gov/mental-health/schizophr...,Mental Illness,['Schizophrenia is a serious brain disorder th...
4,How does schizophrenia impact daily functioning?,"It affects work, socialization, and completing...",https://www.samhsa.gov/mental-health/schizophr...,Mental Illness,['Schizophrenia is a serious brain disorder th...


In [ ]:
results = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Generating Responses"):
    question = row['Question']
    ground_truth = row['Ground_Truth']
    source = row['Source']
    category = row['Category']
    context = row['contexts']

    rag_result = qa_chain.invoke(question)
    rag_response = rag_result['result']

    results.append({
        "Question": question,
        "Ground_Truth": ground_truth,
        "Source": source,
        "Category": category,
        "Context": context,
        "rag_response": rag_response
    })

results_df = pd.DataFrame(results)

In [ ]:
results_df.to_csv("all-mpnet-base-v2-1000-100-responses.csv")

Evaluation

In [ ]:
eval_df = pd.read_csv("all-mpnet-base-v2-1000-100-responses.csv")

In [ ]:
eval_df

,Question,Ground_Truth,Source,Category,Context,rag_response
0,What triggers Seasonal Affective Disorder (SAD)?,"SAD is triggered by seasonal changes, especial...",https://www.samhsa.gov/mental-health/seasonal-...,Mental Illness,"['Seasonal affective disorder, or SAD, is a co...",The context suggests that Seasonal Affective D...
1,What are the health risks of bulimia nervosa?,"Risks include dental decay, dehydration, and s...",https://www.samhsa.gov/mental-health/eating-di...,Mental Illness,['It is common that binge eating will lead to ...,The health risks of bulimia nervosa include ch...
2,How can someone with mental health problems ma...,By finding trustworthy individuals who provide...,https://www.samhsa.gov/mental-health/how-to-ta...,Mental Health,"[""Anyone can experience mental health problems...","Find someone who likes, respects, and trusts y..."
3,What are the risk factors for schizophrenia?,Risk factors include genetic predisposition an...,https://www.samhsa.gov/mental-health/schizophr...,Mental Illness,['Schizophrenia is a serious brain disorder th...,"According to the provided context, biological ..."
4,How does schizophrenia impact daily functioning?,"It affects work, socialization, and completing...",https://www.samhsa.gov/mental-health/schizophr...,Mental Illness,['Schizophrenia is a serious brain disorder th...,Schizophrenia can be extremely disruptive to a...
...,...,...,...,...,...,...
95,How does untreated OCD impact daily life?,It can lead to compulsions that dominate daily...,https://www.samhsa.gov/mental-health/anxiety-d...,Anxiety,"[""counseling can be very effective for ocd . i...","Left untreated, OCD can take over your life."
96,How can parents identify mental health concern...,"By observing signs like extreme sadness, withd...",https://www.samhsa.gov/mental-health/how-to-ta...,Mental Health,"[""As a parent or caregiver, you want the best ...",Parents can identify mental health concerns in...
97,Why do people develop social anxiety disorder?,It often develops due to genetics or negative ...,https://www.samhsa.gov/mental-health/anxiety-d...,Anxiety,['Signs & Symptoms of Social Anxiety Disorder\...,"According to the context, factors such as gene..."
98,What are signs a child may need mental health ...,"Signs include drastic mood swings, extreme fea...",https://www.samhsa.gov/mental-health/how-to-ta...,Mental Health,"[""As a parent or caregiver, you want the best ...","According to the provided context, signs a chi..."


In [ ]:
eval_df = eval_df.to_dict(orient='records')

In [ ]:
# Evaluation prompt
evaluation_prompt = """
### Task Description:
You are provided with an instruction (or query), a response to evaluate, a reference answer that represents an ideal response, and evaluation criteria. Additionally, you are given responses from a RAG model: a RAG response. Your task is to evaluate and compare the RAG response with the reference answer based on the provided criteria.

1. Assess the quality of the response strictly based on the given rubric and criteria (relevance, semantic similarity, accuracy, factual correctness, completeness, clarity, and conciseness), not general impressions.
2. Write short feedback for each response based on the rubric, highlighting strengths and weaknesses for each criterion.
3. Provide feedback in the specified format.
4. Finally, assign an overall score out of 10 for RAG response.


### The instruction to evaluate:
{Question}

### RAG Response:
{rag_response}

### Reference Answer:
{Ground_Truth}

### Rubric for Individual Criteria:
- **Relevance**: How effectively does the response address the specific requirements or intent of the query?
- **Semantic Similarity**: To what extent does the response capture the meaning or intent of the reference answer, even if the phrasing differs?
- **Accuracy**: Does the response provide correct and logically sound information in relation to the query?
- **Factual Correctness**: Are the facts presented in the response verified and consistent with reliable sources or the reference answer?
- **Completeness**: Does the response thoroughly cover all essential aspects of the query without omitting critical information?
- **Clarity**: Is the response expressed in a manner that is straightforward, unambiguous, and easy to comprehend?
- **Conciseness**: Does the response convey the necessary information succinctly, avoiding extraneous details or redundancy?

### Output Format:
Feedback:
- RAG Response:
    - Relevance: {{feedback}}
    - Semantic Similarity: {{feedback}}
    - Accuracy: {{feedback}}
    - Factual Correctness: {{feedback}}
    - Completeness: {{feedback}}
    - Clarity: {{feedback}}
    - Conciseness: {{feedback}}

### Overall Scores:
- RAG Overall Score: {{integer between 1 and 10}}
"""

In [ ]:
import openai

In [ ]:
!pip install openai

In [ ]:
!pip install httpx==0.27.2

In [ ]:
from openai import OpenAI

client = OpenAI(api_key = "YOUR_API_KEY") # Replace with your api key

In [ ]:
import re

def evaluate_responses(instruction, rag_response, reference_answer):
    prompt = evaluation_prompt.format(
        Question=instruction,
        rag_response=rag_response,
        Ground_Truth=reference_answer
    )

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant evaluating model responses."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=1000,
        temperature=0.5
    )

    s = response.choices[0].message.content
    print("Evaluation Output:")
    print(s)

    rag_score = re.search(r'RAG Overall Score: (\d+)', s).group(1).strip()

    return int(rag_score)

evaluations = []
for data in tqdm(eval_df, desc="Evaluating responses", unit="sample"):
    instruction = data["Question"]
    rag_response = data["rag_response"]
    reference_answer = data["Ground_Truth"]


    rag_score = evaluate_responses(
        instruction, rag_response, reference_answer
    )

    evaluations.append({
        "Question": instruction,
        "Ground_Truth": reference_answer,
        "Source": data["Source"],
        "Category": data["Category"],
        "Context": data["Context"],
        "rag_response": rag_response,
        "RAG_Score": rag_score,
    })

Evaluating responses:   1%|          | 1/100 [00:02<04:12,  2.55s/sample]

Evaluation Output:
Feedback:
- RAG Response:
    - Relevance: The response effectively addresses the query by identifying the seasonal changes that trigger SAD. However, it could be more direct in linking the decrease in sunlight specifically to the condition.
    - Semantic Similarity: The response captures the essence of the reference answer, noting the importance of seasonal changes and reduced sunlight, though it elaborates further than necessary.
    - Accuracy: The information provided is accurate, explaining how decreased sunlight can disrupt circadian rhythms and affect serotonin levels.
    - Factual Correctness: The facts presented are consistent with reliable sources regarding SAD, confirming the role of sunlight in triggering the disorder.
    - Completeness: While the response includes relevant details about circadian rhythms and serotonin, it may include more information than necessary for the question, which primarily asks about triggers.
    - Clarity: The response is c

Evaluating responses:   2%|▏         | 2/100 [00:05<04:24,  2.70s/sample]

Evaluation Output:
Feedback:
- RAG Response:
    - Relevance: The response is relevant as it directly addresses the health risks associated with bulimia nervosa, aligning well with the query.
    - Semantic Similarity: The response captures the intent of the reference answer but includes additional details, which slightly diverges from the brevity of the reference.
    - Accuracy: The information provided is accurate; it correctly identifies health risks associated with bulimia nervosa.
    - Factual Correctness: The facts presented are consistent with known health risks related to bulimia nervosa, making it factually correct.
    - Completeness: The response includes several health risks, which adds to its completeness compared to the reference answer, though it could have mentioned other risks for a more comprehensive overview.
    - Clarity: The response is clear and easy to understand, with no ambiguity in the expression of the health risks.
    - Conciseness: While the response is

Evaluating responses:   3%|▎         | 3/100 [00:08<04:39,  2.88s/sample]

...

Evaluation Output:
Feedback:
- RAG Response:
    - Relevance: The response is relevant as it addresses the query about the causes of social anxiety disorder, mentioning genetics and learned behavior.
    - Semantic Similarity: The response captures the essence of the reference answer, although it is slightly more verbose and includes additional details that were not present in the reference.
    - Accuracy: The information provided is accurate, as it correctly identifies genetics and negative social experiences as potential causes of social anxiety disorder.
    - Factual Correctness: The facts presented are consistent with common understandings in psychology regarding social anxiety disorder, aligning with the reference answer.
    - Completeness: The response is somewhat complete but could be improved by mentioning that negative experiences can include more specific examples, as suggested by the reference answer.
    - Clarity: The response is clear and understandable, though the phr

Evaluating responses:  99%|█████████▉| 99/100 [04:12<00:02,  2.55s/sample]

Evaluation Output:
Feedback:
- RAG Response:
    - Relevance: The response is relevant as it directly addresses the signs that indicate a child may need mental health support, aligning well with the query.
    - Semantic Similarity: The response captures the intent of the reference answer but offers a more detailed list rather than a summary, which somewhat diverges from the more concise nature of the reference.
    - Accuracy: The information provided is accurate and aligns with common indicators of mental health issues in children.
    - Factual Correctness: The facts presented are consistent with recognized signs of mental health concerns and are generally supported by reliable sources.
    - Completeness: The response is thorough, covering multiple signs of mental health issues, which adds to its completeness compared to the reference answer.
    - Clarity: The response is clear and easy to understand, with each sign listed in a straightforward manner.
    - Conciseness: While the 

Evaluating responses: 100%|██████████| 100/100 [04:14<00:00,  2.55s/sample]

Evaluation Output:
Feedback:
- RAG Response:
    - Relevance: The response is relevant as it addresses the query about biological factors contributing to mental health conditions.
    - Semantic Similarity: The meaning of the RAG response closely aligns with the reference answer, although it is slightly less formal in phrasing.
    - Accuracy: The information provided is accurate, identifying genes and brain chemistry as biological factors.
    - Factual Correctness: The facts presented are correct and consistent with established knowledge in mental health.
    - Completeness: The response is somewhat limited in scope; while it mentions two important factors, it does not explore other potential biological factors that could be relevant.
    - Clarity: The response is clear and easy to understand, without ambiguity.
    - Conciseness: The response is concise, effectively communicating the necessary information without unnecessary elaboration.

### Overall Scores:
- RAG Overall Score: 7


In [ ]:
import json

with open("evaluation_results_rag_1000_100_sentence_transformers.json", "w") as f:
    json.dump(evaluations, f, indent=4)

In [ ]:
results = pd.DataFrame(evaluations)

In [ ]:
results[['RAG_Score']].describe()

,RAG_Score
count,100.000000
mean,7.180000
std,1.546452
min,1.000000
25%,7.000000
50%,8.000000
75%,8.000000
max,9.000000


In [ ]:
average_rag_score = results['RAG_Score'].mean()

print("Average RAG Score:", average_rag_score)

Average RAG Score: 7.18
